In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest,chi2
import pycountry
import plotly
# !pip install pycountry -q
# !pip install plotly

In [6]:
df = pd.read_csv('ds_salaries_2023.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [7]:
df.shape

(3755, 11)

In [8]:
df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

# **Exploratory Data Analysis**

## **Uni-variate Analysis**

In [9]:
df['remote_ratio']=df['remote_ratio'].map({0:'On-site',50:'Hybrid',100:'Remote'})
df['experience_level']=df['experience_level'].map({'MI':'Mid level','SE':'Senior level','EN':'Entry level','EX':'Executive level'})
df['employment_type']=df['employment_type'].map({'FT':'Full Time','CT':'Contract','FL':'Freelance','PT':'Part Time'})
country_map = {}
for country in pycountry.countries:
    country_map[country.alpha_2] = country.name
# replace values in 'employee_residence' column using dictionary
df['employee_residence'] = df['employee_residence'].replace(country_map)
df['company_location'] = df['company_location'].replace(country_map)

In [19]:
import plotly.io as pio
import plotly.express as px

pio.templates.default = "plotly_dark"
fig = px.bar(df['work_year'].value_counts().reset_index(),
             x='index',
             y='work_year',
             title="Work Year Distribution")
fig.update_xaxes(title="Work Year")
fig.update_yaxes(title="Count")
fig.show()

fig = px.histogram(df, x="experience_level", title="Experience Level Distribution")
fig.update_xaxes(title="Experience Level")
fig.update_yaxes(title="Count")
fig.show()

fig = px.histogram(df, x="employment_type", title="Employment Type Distribution")
fig.update_xaxes(title="Employment Type")
fig.update_yaxes(title="Count")
fig.show()

fig = px.histogram(df, x="remote_ratio", title="Remote Ratio Distribution")
fig.update_xaxes(title="Remote Ratio")
fig.update_yaxes(title="Count")
fig.show()

fig = px.histogram(df, x="company_size", title="Company Size Distribution")
fig.update_xaxes(title="Company Size")
fig.update_yaxes(title="Count")
fig.show()


**Insights from the above plots**

1.   We can observe that the no. of jobs in the field of data increases with as the year goes.
2.  Most of instances available are for the Mid level and Senior level roles
3. Most of the instances available are from the individuals having Full time jobs and others are relatively negligible.
4. Most of the instances available are from the individuals having No remote jobs and more than half of the individuals have Full time remote jobs.
5. Medium size employee candidates are more favoured



In [20]:
fig = px.bar(df['job_title'].value_counts().reset_index(),
             x='index',
             y='job_title',
             title="Job Title Distribution")
fig.update_xaxes(title="Job Title")
fig.update_yaxes(title="Count")
fig.update_xaxes(tickangle=90)  # Rotate x-axis labels
fig.show()

Top most demanding job titles are **Data engineer, Data scientist, Data Analyst, Machine Learning engineer, Analytics Engineer**

In [21]:
fig = px.bar(df['employee_residence'].value_counts().reset_index(),
             x='index',
             y='employee_residence',
             title="Employee Residence Distribution")
fig.update_xaxes(title="Employee Residence")
fig.update_yaxes(title="Count")
fig.update_xaxes(tickangle=90)  # Rotate x-axis labels
fig.show()

Data instances are more biased towards Unites States indicating that most of employees live in United States. This might happen due to insufficient data

In [22]:
fig = px.bar(df['company_location'].value_counts().reset_index(),
             x='index',
             y='company_location',
             title="Company Location Distribution")
fig.update_xaxes(title="Company Location")
fig.update_yaxes(title="Count")
fig.update_xaxes(tickangle=90)  # Rotate x-axis labels
fig.show()

Here the data instances are more biased towards United States indicating that most of the most of the companies are located in United states. This might be due to insufficient data.  

In [24]:
import plotly.figure_factory as ff

fig = ff.create_distplot([df['salary_in_usd']], group_labels=['Salary'], colors=['blue'], show_hist=False, show_rug=False)
fig.update_layout(title="Salary Distribution with KDE")
fig.update_xaxes(title="Salary in USD")
fig.update_yaxes(title="Density")
fig.show()


The salaries are ranging from 0 to $400k. Also we can observe some exceptional values. Since the above distribution is right-skewed , it's better to consider median salary for the future comparisions.

In [25]:
fig = px.box(df, y='salary_in_usd')
fig.update_layout(title="Salary Distribution (Box Plot)")
fig.update_yaxes(title="Salary (in USD)")
fig.show()

In [ ]:
median_salary = df['salary_in_usd'].groupby(df['job_title']).median().nlargest(15).sort_values(ascending=False).reset_index()

## **Bi-Variate Analysis**

In [26]:
fig = px.box(df, x='work_year', y='salary_in_usd', color='work_year', title="Salary Distribution by Work Year")
fig.update_xaxes(title="Work Year")
fig.update_yaxes(title="Salary (in USD)")
fig.show()

Here we can observe that as the work year increases, the salary also increases, which means these two relations are highly correlated

In [27]:
fig = px.box(df, x='experience_level', y='salary_in_usd', color='experience_level', title="Salary Distribution by Experience Level")
fig.update_xaxes(title="Experience Level")
fig.update_yaxes(title="Salary (in USD)")
fig.show()

1. The **highest Median salary** of a executive-level employee is around **200k dollars**.
2. Second highest median salary of a Senior-level employee is around **145k dollars**.
3. Median salary of Mid-level is around **100k dollars**.
4. Median salary of entry-level employee is around **90k dollars**.

In [28]:
fig = px.box(df, x='remote_ratio', y='salary_in_usd', color='remote_ratio', title="Salary Distribution by Remote Ratio")
fig.update_xaxes(title="Remote Ratio")
fig.update_yaxes(title="Salary (in USD)")
fig.show()

Here we observe that the individuals having On-site jobs have the highest  median salary of around **140k dollars** . Also the median salary of individuals having Full Remote jobs is almost same as that of No remote jobs which is around **137k dollars**

In [29]:
fig = px.box(df, x='company_location', y='salary_in_usd', color='company_location', title="Salary Distribution by Company Location")
fig.update_xaxes(title="Company Location")
fig.update_yaxes(title="Salary (in USD)")
fig.update_xaxes(tickangle=90)  # Rotate x-axis labels
fig.show()

Here we can observe that the Companies in **Israel** ,**USA** and **Canada** offer the highest median salaries. Although **Israel** and **Canada** is just 1% of the whole data.

In [30]:
median_salaries = df.groupby('job_title')['salary_in_usd'].median()
top_median_salaries = median_salaries.nlargest(15).sort_values(ascending=False).reset_index()

fig = px.bar(top_median_salaries, x='job_title', y='salary_in_usd', title="Top Median Salaries by Job Title")
fig.update_xaxes(title="Job Titles", tickangle=90)
fig.update_yaxes(title="Salary (in USD)")
fig.show()

From this graph, we can observe that the **executive level** jobs have the highest median salaries .

In [31]:
fig = px.box(df, x='employment_type', y='salary_in_usd', color='employment_type', title="Salary Distribution by Employment Type")
fig.update_xaxes(title="Employment Type")
fig.update_yaxes(title="Salary (in USD)")
fig.show()

The median salary of full time employment is the highest with around 140k dollars. Again the data is highly based, so we cannot conclude which employment type is much better or worth it

## **Data Wrangling**

### **Label Encoding**

In [32]:
df['work_year']=df['work_year'].astype('category')
cat_cols = [col for col in df.columns if df[col].dtype=='O']
df['work_year']=df['work_year'].cat.codes
le=LabelEncoder()
for col in cat_cols:
  df[col]=le.fit_transform(df[col])
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,3,3,2,84,80000,7,85847,66,2,62,0
1,3,2,0,66,30000,19,30000,75,2,70,2
2,3,2,0,66,25500,19,25500,75,2,70,2
3,3,3,2,47,175000,19,175000,11,2,12,1
4,3,3,2,47,120000,19,120000,11,2,12,1


### **Handling outliers**

The interquartile range (IQR) tells you the range of the middle half of your dataset. You can use the IQR to create “fences” around your data and then define outliers as any values that fall outside those fences.


In [33]:
#Inter-quartile range
Q1 = np.percentile(df["salary_in_usd"], 25, interpolation = 'midpoint')
Q2 = np.percentile(df["salary_in_usd"], 50, interpolation = 'midpoint')
Q3 = np.percentile(df["salary_in_usd"], 75, interpolation = 'midpoint')
IQR = Q3 - Q1
lower_fence = Q1 - 1.5 * IQR
upper_fence = Q3 + 1.5 * IQR
outliers=[x for x in df['salary_in_usd'] if x<=lower_fence or x>=upper_fence]
outliers

<ipython-input-33-68a32436c7bb>:2: DeprecationWarning:

the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)

<ipython-input-33-68a32436c7bb>:3: DeprecationWarning:

the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)

<ipython-input-33-68a32436c7bb>:4: DeprecationWarning:

the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)



[342810,
 309400,
 300000,
 342300,
 318300,
 309400,
 300000,
 329500,
 304000,
 353200,
 297300,
 317070,
 423834,
 376080,
 299500,
 297300,
 299500,
 340000,
 310000,
 310000,
 300240,
 300240,
 370000,
 323300,
 299500,
 310000,
 375000,
 318300,
 385000,
 370000,
 314100,
 350000,
 310000,
 300000,
 299500,
 300000,
 300000,
 297300,
 297300,
 310000,
 310000,
 430967,
 300000,
 310000,
 299500,
 300000,
 375000,
 350000,
 315000,
 300000,
 345600,
 300000,
 297500,
 300000,
 300000,
 324000,
 405000,
 380000,
 450000,
 416000,
 325000,
 423000,
 412000]

In [34]:
# ## As there are very less no.of outliers we can drop those values
df.drop(df[df['salary_in_usd']>=324000].index,inplace=True)

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['salary_in_usd'] = scaler.fit_transform(df[['salary_in_usd']])
X = df.drop(['salary','salary_in_usd','salary_currency'],axis=1)  #independent columns
y = df['salary_in_usd']

In [36]:
X.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,3,3,2,84,66,2,62,0
1,3,2,0,66,75,2,70,2
2,3,2,0,66,75,2,70,2
3,3,3,2,47,11,2,12,1
4,3,3,2,47,11,2,12,1


In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 0)
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [39]:
print(model.intercept_)
print(model.coef_)
pred=model.predict(X_test)
predictions = pred.reshape(-1,1)
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
print('MSE : ', mean_squared_error(y_test,predictions))
print('RMSE : ',np.sqrt(mean_squared_error(y_test,predictions)))
# print(pred)

-3.2352008297996866
[ 0.22928461  0.26169586  0.30688063  0.00791996  0.0128318   0.09474933
  0.00269904 -0.08892886]
MSE :  0.7909805982899211
RMSE :  0.8893709002940905
